# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

Please make sure if you are running this notebook in the workspace that you have chosen GPU rather than CPU mode.

In [1]:
# Imports here

import matplotlib.pyplot as plt

import numpy as np
import torch


import torch.nn.functional as F
from torch import nn, optim
from torchvision import datasets, transforms, models
import torchvision.models as models

from PIL import Image
from matplotlib.ticker import FormatStrFormatter


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
# TODO: Define your transforms for the training, validation, and testing sets


#with training data, you typically want to do data argumentation ; rotating it,resizing it, flipping it
#to create this simulated dataset of more images than we acually have
#firstly; it just gives us more data to actually train with
#secondly; it helps the network the network generalize to images that aren't in the training set

#combine these transforms into a pipeline of transform using  (transforms.Compose)
train_transforms =transforms.Compose([transforms.RandomRotation(30),      #random rotation with 30 degree
                                       transforms.RandomResizedCrop(224), #random resize crop; randomly resizing the image and then take a crop from the center of 224 pixel square 
                                       transforms.RandomHorizontalFlip(), #random horizontal flip, it's going to mirror it horizontally 
                                       transforms.ToTensor(),           #change it to a tensor 
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])]) 

#notice one thing to remember is that for testing when you're doing your validation, 
#we don't want to do any of this data augmentation
validation_transforms =transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])]) 

test_transforms =transforms.Compose([transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], 
                                                                 [0.229, 0.224, 0.225])]) 

# TODO: Load the datasets with ImageFolder
#after defining our transforms and then you pass that to dataloder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms) #(transform=train_transform) we pass ourdata directories and our transformer through imagefolder and along with the path to our dataset and that creats a "train_data" object 
validation_data = datasets.ImageFolder(valid_dir, transform=validation_transforms) #so on
test_data = datasets.ImageFolder(test_dir ,transform = test_transforms)    #so on
 



# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True) #DataLoader(we can pass train_data to data folder, this will give us back a generator were we actually can get our images and labels and we give our loaded data to our data loaders to actually get our load our datasets
validationloader = torch.utils.data.DataLoader(validation_data, batch_size =32,shuffle = True)
testloader = torch.utils.data.DataLoader(test_data, batch_size = 20, shuffle = True)

#shuffle=True
#what shuffle does is it randomly shuffles your data every time you start anew epoch
#this's useful because when you're training your network we prefer it 
#the second time it goes through to see your images in a different order
#the third time it goes through to see your images in a different order 
#rather than just learning in the same order every time 
#because then this could introduce weird artifacts in how your network is learning from your data 

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

---
## Define the Model

To define a model for training we'll follow these steps:
1. Load in a pre-trained VGG16 model
2. "Freeze" all the parameters, so the net acts as a fixed feature extractor 
3. Remove the last layer
4. Replace the last layer with a linear classifier of our own

**Freezing simply means that the parameters in the pre-trained model will *not* change during training.**

In [6]:
# TODO: Build and train your network

#load model and use it 
model = models.vgg16(pretrained=True)  #download the pre-trained network, weights and parameter then load it into our model

model # to look at what the architecture of this model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:43<00:00, 12845215.43it/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [7]:
#You can access any layer in a pretrained network by name and (sometimes) number

print(model.classifier[6].in_features) 
print(model.classifier[6].out_features)

4096
1000


In [8]:
# Freeze our features parameters to keep features part static, Freeze training for all "features" layers

for param in model.parameters(): #we go through our parameters in our model
    param.requires_grad = False  #this will do is that when we run our tensor through the model, 
                                #it's not going to calculate the gradients; it is not going to keep track of all these operation 
                               #and also will speed up training because we're not keeping track of these operations for the features  


### Final Classifier Layer

Once you have the pre-trained feature extractor, you just need to modify and/or add to the final, fully-connected classifier layers.

This layer should see as input the number of features produced by the portion of the network that you are not changing, and produce an appropriate number of outputs for the flower classification task.

In [9]:
#replace the last layer in the model classifier group of layers.

# add last linear layer (n_inputs -> 102 flower classes)
# new layers automatically have requires_grad = True

n_inputs = model.classifier[6].in_features
last_layer = nn.Linear(n_inputs, 102)
model.classifier[6] = last_layer 



In [10]:
# check to see that your last layer produces the expected number of outputs

print(model.classifier[6].in_features)
print(model.classifier[6].out_features)


4096
102


In [11]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [12]:
# Define the loss and the optimize

criterion = nn.NLLLoss()
#With a log-softmax output, you want to use the negative log likelihood loss, nn.NLLLoss

optimizer = optim.Adam(model.classifier[6].parameters(), lr=0.003)

#Optimizers require for the last layer which we replace; the parameters to optimize and a learning rate
#optim.Adam this's basically the same as stochastic gradient descent, but it has some nice properties where it uses momentum which speeds up the actual fitting process
#and it also adjust the learning rate for each of the individual parameters in your model

Define [the criterion](http://pytorch.org/docs/master/nn.html#loss-functions) (something like `nn.CrossEntropyLoss` or `nn.NLLLoss`) and [the optimizer](http://pytorch.org/docs/master/optim.html) (typically `optim.SGD` or `optim.Adam`).

In [13]:
model.to(device);

In [14]:
epochs = 12
steps = 0         #tracking the numbel of train steps we do
running_loss = 0 #tracking the loss with running loss
print_every = 5 #how many steps we're going to go before we print out the validation loss
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1   #cumulate steps; every time we go through one of these batches, we're going to increment steps here 
        inputs, labels = inputs.to(device), labels.to(device) #similarly; if you want to run your inputs through your model, you have to make sure that the tensors that you're putting trough your model or on the GPU if your model's on the GPU 
        
        optimizer.zero_grad()            # Clear the gradients, do this because gradients are accumulated.before every training passes

# Forward pass, then backward pass, then update weights

        logps = model.forward(inputs)    #(log probability) to take maximum probability and get minimizing cross entropy;  Forward pass,get our log-probabilities; get our probability by passing in the inputs to a model
        loss = criterion(logps, labels)  #Calculate the loss with the logps and the labels
        loss.backward()                  #we'll do our backwards pass starting from the loss , so we can look at the weight and then we can look at the weight gradients, we can use these gradients in gradint descent to train our network
        optimizer.step()                 #Take an update step and few the new weights

        running_loss += loss.item()      #we can increment are running loss; this way we can keep track of our training loss
        

#after each epochs, after each pass through the training set, then we're going to do a validation pass
#after the above loop complete , run the below code

#we actually want to drop out of the train loop and test our network accuracy and loss on our test dataset
        if steps % print_every == 0:  #then we're going to go into our validation loop
             model.eval()             #turns our model into evaluation inference mode which turns off dropout                                                
            test_loss = 0
            accuracy = 0
            with torch.no_grad():                            # Turn off gradients for validation, saves memory and computations and to speed up this part
                for inputs, labels in testloader:           #get our inputs and labels from testset
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)             #pass images into our model to get our logprobabilities 
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()  #this way allow us to keep track our test loss as we're going through these validation rules
                    
# Calculate accuracy
                    ps = torch.exp(logps)                               #get actual probability distributions from logprobabilities
                    top_p, top_class = ps.topk(1, dim=1)                #give us the top class or prediction class from network; first largest value in our probabilities along the columns

#top_k actually returns a tuple with two tensors
#the first tensor (top_p) actually probability and the second tensor(top_class) are the class indices themselves
#i'm calling (topk) and i'm separating out the probabilities in the classes
#so, we will just use this top class going forward        
         
                    equals = top_class == labels.view(*top_class.shape)            #checking for equality, we can see where our predicted classes match with the true classes from labels; equals should have same shape
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()  #equals has type torch.ByteTensor so we have to convert equals to a float tensor
#basically; we've a running loss of our training loss of our training loss 
#and we've a test loss that we passed our test data through our model and measured the loss in accuracy                
                
                print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            
            running_loss = 0
            
            model.train()                                             #to set model back into train mode which turns on dropout

Epoch 1/3.. Train loss: -1.154.. Test loss: -3.209.. Test accuracy: 0.111
Epoch 1/3.. Train loss: -5.052.. Test loss: -7.258.. Test accuracy: 0.110
Epoch 1/3.. Train loss: -9.405.. Test loss: -11.379.. Test accuracy: 0.089
Epoch 1/3.. Train loss: -14.001.. Test loss: -15.499.. Test accuracy: 0.082
Epoch 1/3.. Train loss: -18.066.. Test loss: -19.590.. Test accuracy: 0.081
Epoch 1/3.. Train loss: -21.815.. Test loss: -23.667.. Test accuracy: 0.084
Epoch 1/3.. Train loss: -28.258.. Test loss: -27.809.. Test accuracy: 0.088
Epoch 1/3.. Train loss: -29.963.. Test loss: -31.886.. Test accuracy: 0.089
Epoch 1/3.. Train loss: -36.792.. Test loss: -35.977.. Test accuracy: 0.074
Epoch 1/3.. Train loss: -40.492.. Test loss: -40.039.. Test accuracy: 0.073
Epoch 1/3.. Train loss: -44.193.. Test loss: -44.164.. Test accuracy: 0.073
Epoch 1/3.. Train loss: -49.241.. Test loss: -48.261.. Test accuracy: 0.079
Epoch 1/3.. Train loss: -52.459.. Test loss: -52.336.. Test accuracy: 0.090
Epoch 1/3.. Train

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes